In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
import pandas as pd
import numpy as np

In [143]:
income_level = pd.read_excel('/content/drive/MyDrive/B A 데이터 모음/00클러스터링/IncomeLevel - 시트1.xlsx')
pop_density = pd.read_csv('/content/drive/MyDrive/B A 데이터 모음/00클러스터링/pop_density.xlsx - 인구밀도_20231109192951.csv')


pop_column_names = ['gu_origin', 'dong_origin' , 'pop_density']
pop_density.columns = pop_column_names
pop_density

,gu_origin,dong_origin,pop_density
0,종로구,사직동,7980
1,종로구,삼청동,2011
2,종로구,부암동,4648
3,종로구,평창동,2121
4,종로구,무악동,24278
...,...,...,...
420,강동구,둔촌1동,308
421,강동구,둔촌2동,28318
422,강동구,암사1동,35974
423,강동구,천호2동,22869


In [144]:
# 분위 정보를 숫자로 변환합니다.
income_level['1Q'] = income_level['1Q'].astype(str)
income_level['2Q'] = income_level['2Q'].astype(str)
income_level['3Q'] = income_level['3Q'].astype(str)
income_level['4Q'] = income_level['3Q'].astype(str)

income_level['1Q'] = income_level['1Q'].str.split('분위').str[0].astype(int)
income_level['2Q'] = income_level['2Q'].str.split('분위').str[0].astype(int)
income_level['3Q'] = income_level['3Q'].str.split('분위').str[0].astype(int)
income_level['4Q'] = income_level['4Q'].str.split('분위').str[0].astype(int)

# # 변환된 분위 숫자들의 평균을 계산합니다.
income_level['average_quantile'] = income_level[['1Q', '2Q', '3Q', '4Q']].mean(axis=1)
income_level


,gu_origin,dong_origin,1Q,2Q,3Q,4Q,average_quantile
0,종로구,청운효자동,8,8,8,8,8.0
1,종로구,사직동,9,9,9,9,9.0
2,종로구,삼청동,7,7,7,7,7.0
3,종로구,부암동,7,7,7,7,7.0
4,종로구,평창동,8,8,8,8,8.0
...,...,...,...,...,...,...,...
420,강동구,성내2동,6,6,6,6,6.0
421,강동구,성내3동,7,7,7,7,7.0
422,강동구,길동,7,7,7,7,7.0
423,강동구,둔촌1동,7,7,7,7,7.0


In [145]:
#중복데이터 확인
print(income_level['dong_origin'].duplicated().sum())
print(pop_density['dong_origin'].duplicated().sum())

1
1


In [146]:
# income_level에서 'dong_origin' 컬럼의 중복된 값 확인
duplicates_income_level = income_level[income_level.duplicated(subset='dong_origin', keep=False)]

# pop_density에서 'dong_origin' 컬럼의 중복된 값 확인
duplicates_pop_density = pop_density[pop_density.duplicated(subset='dong_origin', keep=False)]

# 중복된 값 출력
print("중복된 값 in income_level:")
print(duplicates_income_level)

print("\n중복된 값 in pop_density:")
print(duplicates_pop_density)

중복된 값 in income_level:
    gu_origin dong_origin  1Q  2Q  3Q  4Q  average_quantile
332       관악구         신사동   6   6   6   6               6.0
358       강남구         신사동   9   9   9   9               9.0

중복된 값 in pop_density:
    gu_origin dong_origin  pop_density
329       관악구         신사동        40563
358       강남구         신사동         9284


In [147]:
common_keys = set(income_level['gu_origin']).intersection(set(pop_density['gu_origin']))
income_level_unique_keys = set(income_level['gu_origin']) - common_keys
pop_density_unique_keys = set(pop_density['gu_origin']) - common_keys

print(f"Common keys: {len(common_keys)}")
print(f"Income_level unique keys: {income_level_unique_keys}")
print(f"Pop_density unique keys: {pop_density_unique_keys}")

Common keys: 25
Income_level unique keys: set()
Pop_density unique keys: set()


In [148]:
common_keys = set(income_level['dong_origin']).intersection(set(pop_density['dong_origin']))
income_level_unique_keys = set(income_level['dong_origin']) - common_keys
pop_density_unique_keys = set(pop_density['dong_origin']) - common_keys

print(f"Common keys: {len(common_keys)}")
print(f"Income_level unique keys: {income_level_unique_keys}")
print(f"Pop_density unique keys: {pop_density_unique_keys}")

Common keys: 423
Income_level unique keys: {'항동'}
Pop_density unique keys: {'소계'}


In [149]:
# dong 기준으로 합친다

Income_popdensity = pd.merge(income_level, pop_density, on=['gu_origin','dong_origin'], how='inner')
Income_popdensity

,gu_origin,dong_origin,1Q,2Q,3Q,4Q,average_quantile,pop_density
0,종로구,청운효자동,8,8,8,8,8.0,5051
1,종로구,사직동,9,9,9,9,9.0,7980
2,종로구,삼청동,7,7,7,7,7.0,2011
3,종로구,부암동,7,7,7,7,7.0,4648
4,종로구,평창동,8,8,8,8,8.0,2121
...,...,...,...,...,...,...,...,...
419,강동구,성내2동,6,6,6,6,6.0,37591
420,강동구,성내3동,7,7,7,7,7.0,33692
421,강동구,길동,7,7,7,7,7.0,21367
422,강동구,둔촌1동,7,7,7,7,7.0,308


In [150]:
# Income_level에 항동이 있는데 인구밀도에는 없다
# 근데 왜 row 수가 늘지??
Income_popdensity = Income_popdensity[Income_popdensity['dong_origin'] != '항동']
Income_popdensity

,gu_origin,dong_origin,1Q,2Q,3Q,4Q,average_quantile,pop_density
0,종로구,청운효자동,8,8,8,8,8.0,5051
1,종로구,사직동,9,9,9,9,9.0,7980
2,종로구,삼청동,7,7,7,7,7.0,2011
3,종로구,부암동,7,7,7,7,7.0,4648
4,종로구,평창동,8,8,8,8,8.0,2121
...,...,...,...,...,...,...,...,...
419,강동구,성내2동,6,6,6,6,6.0,37591
420,강동구,성내3동,7,7,7,7,7.0,33692
421,강동구,길동,7,7,7,7,7.0,21367
422,강동구,둔촌1동,7,7,7,7,7.0,308


In [151]:
seoul_code = pd.read_excel('/content/drive/MyDrive/B A 데이터 모음/00클러스터링/통행량_수단전체/서울행정구역_코드.xlsx', sheet_name='Sheet1', header=0)

In [153]:
seoul_code_columns = ['gu_origin',	'dong_origin','TAZ'	,'10code',	'Node_id']
seoul_code.columns = seoul_code_columns
seoul_code

,gu_origin,dong_origin,TAZ,10code,Node_id
0,종로구,삼청동,1,1111054000,586744
1,종로구,가회동,2,1111060000,586741
2,종로구,청운효자동,3,1111051500,586743
3,종로구,부암동,4,1111055000,586745
4,종로구,평창동,5,1111056000,586746
...,...,...,...,...,...
419,강동구,성내2동,420,1174065000,586632
420,강동구,성내3동,421,1174066000,586631
421,강동구,둔촌1동,422,1174069000,586629
422,강동구,둔촌2동,423,1174070000,586633


In [159]:
common_keys = set(Income_popdensity['dong_origin']).intersection(set(seoul_code['dong_origin']))
income_level_unique_keys = set(Income_popdensity['dong_origin']) - common_keys
pop_density_unique_keys = set(seoul_code['dong_origin']) - common_keys

print(f"Common keys: {len(common_keys)}")
print(f"Income_level unique keys: {income_level_unique_keys}")
print(f"Pop_density unique keys: {pop_density_unique_keys}")

Common keys: 416
Income_level unique keys: {'종로5.6가동', '중계2.3동', '상계6.7동', '상계3.4동', '종로1.2.3.4가동', '면목3.8동', '금호2.3가동'}
Pop_density unique keys: {'종로5·6가동', '상계3·4동', '종로1·2·3·4가동', '상계6·7동', '면목3·8동', '중계2·3동', '금호2·3가동'}


In [160]:
Income_popdensity_beforechage = ['종로5.6가동', '중계2.3동', '상계6.7동', '상계3.4동', '종로1.2.3.4가동', '면목3.8동', '금호2.3가동']
Pop_density_afterchage = ['종로5·6가동', '중계2·3동', '상계6·7동', '상계3·4동', '종로1·2·3·4가동', '면목3·8동',  '금호2·3가동']


for i in range(len(Income_popdensity_beforechage)):
  Income_popdensity['dong_origin'] = Income_popdensity['dong_origin'].str.replace(Income_popdensity_beforechage[i], Pop_density_afterchage[i])


<ipython-input-160-a982d12105d2>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  Income_popdensity['dong_origin'] = Income_popdensity['dong_origin'].str.replace(Income_popdensity_beforechage[i], Pop_density_afterchage[i])


In [161]:
common_keys = set(Income_popdensity['dong_origin']).intersection(set(seoul_code['dong_origin']))
income_level_unique_keys = set(Income_popdensity['dong_origin']) - common_keys
pop_density_unique_keys = set(seoul_code['dong_origin']) - common_keys

print(f"Common keys: {len(common_keys)}")
print(f"Income_level unique keys: {income_level_unique_keys}")
print(f"Pop_density unique keys: {pop_density_unique_keys}")

Common keys: 423
Income_level unique keys: set()
Pop_density unique keys: set()


In [170]:
# 존 코드를 합친다

Income_popdensity_code = pd.merge(Income_popdensity, seoul_code, on=['gu_origin','dong_origin'], how='inner')
Income_popdensity_code

,gu_origin,dong_origin,1Q,2Q,3Q,4Q,average_quantile,pop_density,TAZ,10code,Node_id
0,종로구,청운효자동,8,8,8,8,8.0,5051,3,1111051500,586743
1,종로구,사직동,9,9,9,9,9.0,7980,6,1111053000,586735
2,종로구,삼청동,7,7,7,7,7.0,2011,1,1111054000,586744
3,종로구,부암동,7,7,7,7,7.0,4648,4,1111055000,586745
4,종로구,평창동,8,8,8,8,8.0,2121,5,1111056000,586746
...,...,...,...,...,...,...,...,...,...,...,...
419,강동구,성내2동,6,6,6,6,6.0,37591,420,1174065000,586632
420,강동구,성내3동,7,7,7,7,7.0,33692,421,1174066000,586631
421,강동구,길동,7,7,7,7,7.0,21367,424,1174068500,586635
422,강동구,둔촌1동,7,7,7,7,7.0,308,422,1174069000,586629


In [171]:
selected_colunms = ['gu_origin','dong_origin','10code','average_quantile','pop_density']
Income_popdensity_code= Income_popdensity_code[selected_colunms]
Income_popdensity_code_columns = ['gu',	'dong','code','income_level','pop_density']
Income_popdensity_code.columns = Income_popdensity_code_columns

Income_popdensity_code

,gu,dong,code,income_level,pop_density
0,종로구,청운효자동,1111051500,8.0,5051
1,종로구,사직동,1111053000,9.0,7980
2,종로구,삼청동,1111054000,7.0,2011
3,종로구,부암동,1111055000,7.0,4648
4,종로구,평창동,1111056000,8.0,2121
...,...,...,...,...,...
419,강동구,성내2동,1174065000,6.0,37591
420,강동구,성내3동,1174066000,7.0,33692
421,강동구,길동,1174068500,7.0,21367
422,강동구,둔촌1동,1174069000,7.0,308


In [172]:
address = "/content/drive/MyDrive/B A 데이터 모음/00클러스터링/Preprocessed_datasets/"
Income_popdensity_code.to_csv(path_or_buf = address + "Income_Pdensity_Zoncode.csv")